In [5]:
%%javascript
IPython.OutputArea.prototype._should_scroll = function(lines) {
    return false;
}

<IPython.core.display.Javascript object>

In [6]:
#######################################################
#                 constants for the evaluators
#######################################################

MIN_MEASUREMENT_NUMBER = 5

In [7]:
#######################################################
#                 in-phase evaluator - mprtp2
#######################################################

# mprtp 2 - equal RTT, changing bw
# mprtp 8 - equal RTT, equal bw
# mprtp 9 - different RTT, equal bw

import pandas as pd
import numpy as np

class PDF(object):
    def __init__(self, pdf, size=(400,200)):
        self.pdf = pdf
        self.size = size

    def _repr_html_(self):
        return '<iframe src={0} width={1[0]} height={1[1]}></iframe>'.format(self.pdf, self.size)

    def _repr_latex_(self):
        return r'\includegraphics[width=1.0\textwidth]{{{0}}}'.format(self.pdf)

packets_colnames = ["Tracked NTP",
                    "Sequence Number",
                    "Timestamp",
                    "SSRC",
                    "Payload Type",
                    "Payload Size",
                    "Subflow ID",
                    "Subflow Sequence Number",
                    "Header Size",
                    "Protect Begin",
                    "Protect End",
                    "Marker",
                   ]

class Subflow:
    def __init__(self, channel, path, subflow_id):
        self._path = path
        self._subflow_id = subflow_id
        self._df = pd.DataFrame()
        sr = os.path.join(path, 'rtpsubflow_'+ str(subflow_id) +'_sr.csv')
        pathbw = os.path.join(path, 'subflow_'+ str(subflow_id) +'_pathbw.csv')
        qmd = os.path.join(path, 'rtpsubflow_'+ str(subflow_id) +'_qmd.csv')
        self._df = pd.concat([
            pd.read_csv(sr, index_col=None, header=None, names=['Sending Rate', 'FEC Rate']),
            pd.read_csv(pathbw, index_col=None, header=None, names=['Capacity']),
            pd.read_csv(qmd, index_col=None, header=None, names=['Queue Delay']),
        ], axis=1)

        snd_csv = os.path.join(path, 'snd_packets_' + channel + '_s'+ str(subflow_id) +'.csv')
        snd_packets = pd.read_csv(snd_csv, index_col=None, header=None, names=packets_colnames)
        snd_packets = snd_packets[snd_packets["Payload Type"] == 96]
        
        ply_csv = os.path.join(path, 'rcv_packets_' + channel + '_s'+ str(subflow_id) +'.csv')
        ply_packets = pd.read_csv(ply_csv, index_col=None, header=None, names=packets_colnames)
        ply_packets = ply_packets[ply_packets["Payload Type"] == 96]
        
        self._df['SR'] = self._df.apply (lambda row: (row['Sending Rate'] + row['FEC Rate']) // 125, axis=1)
        self._df['ABU'] = self._df.apply (lambda row: ((row['Sending Rate'] + row['FEC Rate']) // 125) / row['Capacity'], axis=1)        
        self._df['QMD'] = self._df.apply (lambda row: row['Queue Delay'] // 1000, axis=1)
        self._lr = 1.0 - float(len(ply_packets)) / float(len(snd_packets))
        self._lrn = len(snd_packets) - len(ply_packets)
        gp = os.path.join(path, 'rtpsubflow_'+ str(subflow_id) +'_gp_avg.csv')
        with open(gp, 'r') as content_file:
            self._gp = float(content_file.read()) / 125
        
    @property
    def sr(self):
        return self._sr

    def get_summary(self):
        return {
            'SR': self._df['SR'].mean(),
            'ABU': self._df["ABU"].mean(),
            'QMD': self._df['QMD'].mean(),
            'LR': self._lr,
            'LRN': self._lrn,            
            'GP': self._gp
        }
        
    
def collect_subflows(channel, path, subflows_num):
    subflows = [Subflow(channel, path, subflow_id) for subflow_id in range(1, subflows_num + 1)]
    return subflows

    
import os
from os import listdir
from os.path import isfile, join

configs = [
{
    'base_path': "mprtp2", "test_num": 2,
#     'base_path': "mprtp8", "test_num": 8
    'debug': False,
#     'debug': True,
    'channel': "1"
},
]

for config in configs:
    print(config)
    for path,dirs,files in os.walk(config['base_path']):
        if len(dirs) < 1:
            continue
        final_table = pd.DataFrame()
    #     for subflows_num in [5]:
        for subflows_num in [1, 2, 3, 5]:
    #         print("number of subflows", subflows_num)
            suffix = "_" + str(subflows_num)
            metrics = {
                'SR': [],
                'ABU': [],
                'LR': [],
                'LRN': [],            
                'QMD': [],
                'GP': [],
            }
            subjects = []
            for directory in dirs:
                if not directory.endswith(suffix):
                    continue
                try:
                    subflows = collect_subflows(config['channel'], config['base_path'] + '/' + directory, subflows_num)
                except:
                    print("There is an exception occured at:", directory)
                
                abs_ABU = sum(subflow.get_summary()['ABU'] for subflow in subflows)
                abs_LR = sum(subflow.get_summary()['LR'] for subflow in subflows)
                abs_GP = sum(subflow.get_summary()['GP'] for subflow in subflows)
                abs_QMD = sum(subflow.get_summary()['QMD'] for subflow in subflows)
                abs_LRN = sum(subflow.get_summary()['LRN'] for subflow in subflows)
                subjects.append((directory, abs_ABU, abs_LR, abs_GP, abs_QMD, abs_LRN))
                
                SR = sum(subflow.get_summary()['SR'] for subflow in subflows) 
                ABU = sum(subflow.get_summary()['ABU'] for subflow in subflows) / float(subflows_num)
                LR = sum(subflow.get_summary()['LR'] for subflow in subflows) / float(subflows_num)
                LRN = sum(subflow.get_summary()['LRN'] for subflow in subflows)           
                QMD = sum(subflow.get_summary()['QMD'] for subflow in subflows) / float(subflows_num)
                GP = sum(subflow.get_summary()['GP'] for subflow in subflows)
                metrics['SR'].append(SR)
                metrics['ABU'].append(ABU)            
                metrics['LR'].append(LR)
                metrics['LRN'].append(LRN)            
                metrics['QMD'].append(QMD)
                metrics['GP'].append(GP)

            # Selection method
            # setup sets for the max 10 based on criteria (LR, ABU, and verifications) and filter out the 
            # intersection of the weakers

            # Display parameters
            if config['debug']:
                index = 1
                if (len(subjects) < MIN_MEASUREMENT_NUMBER):
                    print("for measuring ', subflows_num, 'subflows, \
                          it does not have the necessary minimum number of measurements")
    #             sorted_list = sorted(subjects, key=lambda x: x[1], reverse=True)
                sorted_list = sorted(subjects, key=lambda x: x[4], reverse=False)
                for item in sorted_list:
                    print("Rank:", index)
                    print("Directory, ABU, LR, GP, QMD, LRN: ", item)
                    pdf = PDF(config['base_path'] + '/' + item[0] + '/' + 'mprtp' + config['test_num'] 
                              + '_aggr_FRACTaL_50ms_0ms.pdf')
                    display(pdf)
                    index += 1
                print('-' * 20)

            subflow_col = (str(subflows_num) + ' Subflows') if 1 < subflows_num else '1 Subflow'
            final_table[subflow_col] = list(range(7))
            SRs = np.asarray(metrics['SR'])
            ABUs = np.asarray(metrics['ABU'])
            LRs = np.asarray(metrics['LR'])
            LRNs = np.asarray(metrics['LRN'])
            QMDs = np.asarray(metrics['QMD'])
            GPs = np.asarray(metrics['GP'])
            
            try:
                final_table[subflow_col][0] = str(int(GPs.mean())) + ' ± ' + str(int(GPs.std()))
                final_table[subflow_col][1] = str(int(ABUs.mean() * 100)) + '% ± ' + str(int(ABUs.std() * 100) ) + '%'
                final_table[subflow_col][2] = str(int(LRs.mean() * 100) ) + '% ± ' + str(int(LRs.std() * 100)) + '%'
                final_table[subflow_col][3] = str(int(LRNs.mean())) + ' ± ' + str(int(LRNs.std()))        
                final_table[subflow_col][4] = str(round(QMDs.mean(), 2)) + ' ± ' + str(round(QMDs.std(), 2))
                final_table[subflow_col][5] = str(int(SRs.mean())) + ' ± ' + str(int(SRs.std()) )
                final_table[subflow_col][6] = str(len(metrics['SR']))
            except:
                print("Problem in final table", directory)

        final_table = final_table.rename({
            0: 'GP [kbps]',
            1: 'ABU [%]', 
            2: 'LR [%]', 
            3: 'LRN',
            4: 'QMD [ms]',
            5: 'SR [kbps]',
            6: 'Samples'}, axis='index')
    #     display(final_table)
        display(final_table.loc[['GP [kbps]', 'ABU [%]', 'LR [%]', 'QMD [ms]'], :])


{'debug': False, 'test_num': 2, 'base_path': 'mprtp2', 'channel': '1'}


/usr/local/lib/python2.7/site-packages/ipykernel_launcher.py:179: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


,1 Subflow,2 Subflows,3 Subflows,5 Subflows
GP [kbps],372 ± 16,638 ± 38,916 ± 19,1462 ± 19
ABU [%],74% ± 2%,65% ± 2%,63% ± 0%,60% ± 0%
LR [%],0% ± 0%,0% ± 0%,0% ± 0%,0% ± 0%
QMD [ms],111.9 ± 1.14,103.56 ± 0.88,102.39 ± 0.47,101.68 ± 0.11


In [12]:
#######################################################
#                 out-of-phase evaluator - mprtp4
#######################################################

from io import StringIO
import numpy as np
import pandas as pd
from math import log
import math

import pandas as pd
pd.set_option('display.height', 1000)
pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)
pd.set_option('display.width', 1000)

def extract_dataframe(filename, identifier):
    header = False
    content = []
    identifier_length = len(identifier)
    with open(filename) as f:
        lines = f.readlines()
        for line in lines:
            if identifier not in line:
                continue
            stat_line = line[identifier_length:]
            if "Subflow ID" in stat_line:
                if header is True:
                    continue
                header = True
            content.append(stat_line)
    if len(content) < 1:
        return None
    TESTDATA = StringIO('\n'.join(map(unicode,content)))
    return pd.read_csv(TESTDATA)


class PDF(object):
    def __init__(self, pdf, size=(400,200)):
        self.pdf = pdf
        self.size = size

    def _repr_html_(self):
        return '<iframe src={0} width={1[0]} height={1[1]}></iframe>'.format(self.pdf, self.size)

    def _repr_latex_(self):
        return r'\includegraphics[width=1.0\textwidth]{{{0}}}'.format(self.pdf)


class Subflow:
    def __init__(self, path, stats_df, subflow_id):
        self._path = path
        self._subflow_id = subflow_id
        self._stat = stats_df[stats_df["Subflow ID"] == subflow_id]

        sr = os.path.join(path, 'rtpsubflow_'+ str(subflow_id) +'_sr.csv')
        pathbw = os.path.join(path, 'subflow_'+ str(subflow_id) +'_pathbw.csv')
        qmd = os.path.join(path, 'rtpsubflow_'+ str(subflow_id) +'_qmd.csv')
        self._df = pd.concat([
            pd.read_csv(sr, index_col=None, header=None, names=['Sending Rate', 'FEC Rate']),
            pd.read_csv(pathbw, index_col=None, header=None, names=['Capacity']),
            pd.read_csv(qmd, index_col=None, header=None, names=['Queue Delay']),
        ], axis=1)

        snd_csv = os.path.join(path, 'snd_packets_1_s'+ str(subflow_id) +'.csv')
        snd_packets = pd.read_csv(snd_csv, index_col=None, header=None, names=packets_colnames)
        snd_packets = snd_packets[snd_packets["Payload Type"] == 96]
        
        ply_csv = os.path.join(path, 'rcv_packets_1_s'+ str(subflow_id) +'.csv')
        ply_packets = pd.read_csv(ply_csv, index_col=None, header=None, names=packets_colnames)
        ply_packets = ply_packets[ply_packets["Payload Type"] == 96]
        
        self._df['SR'] = self._df.apply (lambda row: (row['Sending Rate'] + row['FEC Rate']) // 125, axis=1)
        self._df['ABU'] = self._df.apply (lambda row: ((row['Sending Rate'] + row['FEC Rate']) // 125) / row['Capacity'], axis=1)        
        self._df['QMD'] = self._df.apply (lambda row: row['Queue Delay'] // 1000, axis=1)
        self._lr = 1.0 - float(len(ply_packets)) / float(len(snd_packets))
        self._lrn = len(snd_packets) - len(ply_packets)
        gp = os.path.join(path, 'rtpsubflow_'+ str(subflow_id) +'_gp_avg.csv')
        with open(gp, 'r') as content_file:
            self._gp = float(content_file.read()) / 125
        
    def get_summary(self):
        return {
            'SR': self._df['SR'].mean(),
            'ABU': self._df["ABU"].mean(),
            'QMD': self._df['QMD'].mean(),
            'LR': self._lr,
            'LRN': self._lrn,            
            'GP': self._gp,
            'TE': self._stat["Target Error"].mean()
        }

# The variance around the target is not significantly higher than in case of one subflow
# the ratio stability

base_path = "mprtp4"
debug = False
# debug = True
for path,dirs,files in os.walk(base_path):
    if len(dirs) < 1:
        continue
    final_table = pd.DataFrame()
#     for subflows_num in [1]:
    for subflows_num in [1, 2]:
        suffix = "_" + str(subflows_num)
        metrics = {
            'SR': [],
            'ABU': [],
            'LR': [],
            'LRN': [],            
            'QMD': [],
            'GP': [],
            'TE': [],
        }
        subjects = []
        for directory in dirs:
            if not directory.endswith(suffix):
                continue
            filename = os.path.join(base_path, directory, "mprtpflow_4_" + str(subflows_num) + "-snd.log")
            stats_df = extract_dataframe(filename, "Stat:")
#             display(stats_df)
            stats_df["Target Error"] = abs(stats_df["Allocated Target [kbps]"] - stats_df["Sending Rate [kbps]"]) / stats_df["Allocated Target [kbps]"]
            subflows = []
            for subflow_id in range(1, subflows_num + 1):
                subflows.append(Subflow(os.path.join(base_path, directory), stats_df, subflow_id))
            
            abs_ABU = sum(subflow.get_summary()['ABU'] for subflow in subflows)
            abs_LR = sum(subflow.get_summary()['LR'] for subflow in subflows)
            abs_GP = sum(subflow.get_summary()['GP'] for subflow in subflows)
            abs_QMD = sum(subflow.get_summary()['QMD'] for subflow in subflows)
            abs_TE = sum(subflow.get_summary()['TE'] for subflow in subflows)
            subjects.append((directory, abs_ABU, abs_LR, abs_GP, abs_QMD, abs_TE))
            
            SR = sum(subflow.get_summary()['SR'] for subflow in subflows) 
            ABU = sum(subflow.get_summary()['ABU'] for subflow in subflows) / float(subflows_num)
            LR = sum(subflow.get_summary()['LR'] for subflow in subflows) / float(subflows_num)
            LRN = sum(subflow.get_summary()['LRN'] for subflow in subflows)           
            QMD = sum(subflow.get_summary()['QMD'] for subflow in subflows) / float(subflows_num)
            GP = sum(subflow.get_summary()['GP'] for subflow in subflows)
            TE = sum(subflow.get_summary()['TE'] for subflow in subflows) / float(subflows_num)            
            metrics['SR'].append(SR)
            metrics['ABU'].append(ABU)            
            metrics['LR'].append(LR)
            metrics['LRN'].append(LRN)            
            metrics['QMD'].append(QMD)
            metrics['GP'].append(GP)
            metrics['TE'].append(TE)
        
        # Display parameters
        if debug:
            index = 1
            if (len(subjects) < MIN_MEASUREMENT_NUMBER):
                print("for measuring ', subflows_num, 'subflows, \
                      it does not have the necessary minimum number of measurements")
#             sorted_list = sorted(subjects, key=lambda x: x[1], reverse=True)
            sorted_list = sorted(subjects, key=lambda x: x[4], reverse=False)
            for item in sorted_list:
                print("Rank:", index)
                print("Directory, ABU, LR, GP, QMD, TE: ", item)
                pdf = PDF(base_path + '/' + item[0] + '/' + 'mprtp4_aggr_FRACTaL_50ms_0ms.pdf')
                display(pdf)
                index += 1
            print('-' * 20)
            
        subflow_col = (str(subflows_num) + ' Subflows') if 1 < subflows_num else '1 Subflow'
        final_table[subflow_col] = list(range(7))
        SRs = np.asarray(metrics['SR'])
        ABUs = np.asarray(metrics['ABU'])
        LRs = np.asarray(metrics['LR'])
        LRNs = np.asarray(metrics['LRN'])
        QMDs = np.asarray(metrics['QMD'])
        GPs = np.asarray(metrics['GP'])
        TEs = np.asarray(metrics['TE'])

        final_table[subflow_col][0] = str(int(GPs.mean())) + ' ± ' + str(int(GPs.std()))
        final_table[subflow_col][1] = str(int(ABUs.mean() * 100)) + '% ± ' + str(int(ABUs.std() * 100) ) + '%'
        final_table[subflow_col][2] = str(int(LRs.mean() * 100) ) + '% ± ' + str(int(LRs.std() * 100)) + '%'
        final_table[subflow_col][3] = str(int(LRNs.mean())) + ' ± ' + str(int(LRNs.std()))        
        final_table[subflow_col][4] = str(round(QMDs.mean(), 2)) + ' ± ' + str(round(QMDs.std(), 2))
        final_table[subflow_col][5] = str(int(SRs.mean())) + ' ± ' + str(int(SRs.std()) )
        final_table[subflow_col][6] = str(int(TEs.mean() * 100)) + '% ± ' + str(int(TEs.std() * 100) ) + '%'
        final_table[subflow_col][7] = str(len(metrics['SR']))
        
    final_table = final_table.rename({
        0: 'GP [kbps]',
        1: 'ABU [%]', 
        2: 'LR [%]', 
        3: 'LRN',
        4: 'QMD [ms]',
        5: 'SR [kbps]',
        6: 'TE [%]',
        7: 'Samples'}, axis='index')
#     display(final_table)
    display(final_table.loc[['GP [kbps]', 'TE [%]', 'LR [%]', 'QMD [ms]'], :])



/usr/local/lib/python2.7/site-packages/ipykernel_launcher.py:174: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


,1 Subflow,2 Subflows
GP [kbps],420 ± 33,631 ± 18
TE [%],16% ± 1%,16% ± 1%
LR [%],1% ± 0%,0% ± 0%
QMD [ms],112.52 ± 2.09,103.64 ± 0.98


In [22]:
#######################################################
#      2 bottleneck, one shared - mprtp6
#######################################################
# TODO!
import pandas as pd
import numpy as np

class PDF(object):
    def __init__(self, pdf, size=(400,200)):
        self.pdf = pdf
        self.size = size

    def _repr_html_(self):
        return '<iframe src={0} width={1[0]} height={1[1]}></iframe>'.format(self.pdf, self.size)

    def _repr_latex_(self):
        return r'\includegraphics[width=1.0\textwidth]{{{0}}}'.format(self.pdf)

packets_colnames = ["Tracked NTP",
                    "Sequence Number",
                    "Timestamp",
                    "SSRC",
                    "Payload Type",
                    "Payload Size",
                    "Subflow ID",
                    "Subflow Sequence Number",
                    "Header Size",
                    "Protect Begin",
                    "Protect End",
                    "Marker",
                   ]

class Subflow:
    def __init__(self, channel, path, subflow_id):
        self._path = path
        self._subflow_id = subflow_id
        self._df = pd.DataFrame()
        sr = os.path.join(path, 'rtpsubflow_'+ str(subflow_id) +'_sr.csv')
        pathbw = os.path.join(path, 'subflow_'+ str(subflow_id) +'_pathbw.csv')
        qmd = os.path.join(path, 'rtpsubflow_'+ str(subflow_id) +'_qmd.csv')
        self._df = pd.concat([
            pd.read_csv(sr, index_col=None, header=None, names=['Sending Rate', 'FEC Rate']),
            pd.read_csv(pathbw, index_col=None, header=None, names=['Capacity']),
            pd.read_csv(qmd, index_col=None, header=None, names=['Queue Delay']),
        ], axis=1)

        snd_csv = os.path.join(path, 'snd_packets_' + channel + '_s'+ str(subflow_id) +'.csv')
        snd_packets = pd.read_csv(snd_csv, index_col=None, header=None, names=packets_colnames)
        snd_packets = snd_packets[snd_packets["Payload Type"] == 96]
        
        ply_csv = os.path.join(path, 'rcv_packets_' + channel + '_s'+ str(subflow_id) +'.csv')
        ply_packets = pd.read_csv(ply_csv, index_col=None, header=None, names=packets_colnames)
        ply_packets = ply_packets[ply_packets["Payload Type"] == 96]
        
        self._df['SR'] = self._df.apply (lambda row: (row['Sending Rate'] + row['FEC Rate']) // 125, axis=1)
        self._df['ABU'] = self._df.apply (lambda row: ((row['Sending Rate'] + row['FEC Rate']) // 125) / row['Capacity'], axis=1)        
        self._df['QMD'] = self._df.apply (lambda row: row['Queue Delay'] // 1000, axis=1)
        self._lr = 1.0 - float(len(ply_packets)) / float(len(snd_packets))
        self._lrn = len(snd_packets) - len(ply_packets)
        gp = os.path.join(path, 'rtpsubflow_'+ str(subflow_id) +'_gp_avg.csv')
        with open(gp, 'r') as content_file:
            self._gp = float(content_file.read()) / 125
        
    @property
    def sr(self):
        return self._sr

    def get_summary(self):
        return {
            'SR': self._df['SR'].mean(),
            'ABU': self._df["ABU"].mean(),
            'QMD': self._df['QMD'].mean(),
            'LR': self._lr,
            'LRN': self._lrn,            
            'GP': self._gp
        }
        

class SingleFlow:
    def __init__(self, channel, path, flow_id):
        self._path = path
        self._flow_id = flow_id
        self._df = pd.DataFrame()
        sr = os.path.join(path, 'flow_'+ str(flow_id) +'_sr.csv')
        pathbw = os.path.join(path, 'flows_pathbw.csv')
        qmd = os.path.join(path, 'flow_'+ str(flow_id) +'_qmd.csv')
        self._df = pd.concat([
            pd.read_csv(sr, index_col=None, header=None, names=['Sending Rate', 'FEC Rate']),
            pd.read_csv(pathbw, index_col=None, header=None, names=['Capacity']),
            pd.read_csv(qmd, index_col=None, header=None, names=['Queue Delay']),
        ], axis=1)

        snd_csv = os.path.join(path, 'snd_packets_' + channel + '.csv')
        snd_packets = pd.read_csv(snd_csv, index_col=None, header=None, names=packets_colnames)
        snd_packets = snd_packets[snd_packets["Payload Type"] == 96]
        
        ply_csv = os.path.join(path, 'rcv_packets_' + channel +'.csv')
        ply_packets = pd.read_csv(ply_csv, index_col=None, header=None, names=packets_colnames)
        ply_packets = ply_packets[ply_packets["Payload Type"] == 96]
        
        self._df['SR'] = self._df.apply (lambda row: (row['Sending Rate'] + row['FEC Rate']) // 125, axis=1)
        self._df['ABU'] = self._df.apply (lambda row: ((row['Sending Rate'] + row['FEC Rate']) // 125) / row['Capacity'], axis=1)        
        self._df['QMD'] = self._df.apply (lambda row: row['Queue Delay'] // 1000, axis=1)
        self._lr = 1.0 - float(len(ply_packets)) / float(len(snd_packets))
        self._lrn = len(snd_packets) - len(ply_packets)
        gp = os.path.join(path, 'flow_'+ str(flow_id) +'_gp_avg.csv')
        with open(gp, 'r') as content_file:
            self._gp = float(content_file.read()) / 125
        
    @property
    def sr(self):
        return self._sr

    def get_summary(self):
        return {
            'SR': self._df['SR'].mean(),
            'ABU': self._df["ABU"].mean(),
            'QMD': self._df['QMD'].mean(),
            'LR': self._lr,
            'LRN': self._lrn,            
            'GP': self._gp
        }
        
def collect_subflows(channel, path, subflows_num):
    subflows = [Subflow(channel, path, subflow_id) for subflow_id in range(1, subflows_num + 1)]
    return subflows

    
import os
from os import listdir
from os.path import isfile, join

configs = [
{
#     'base_path': "mprtp6",
    'base_path': "mprtp5",
    'debug': False,
#     'debug': True,
    'mp_channel': "2",
    'single_channel': "1",
}, 
]

for config in configs:
    print(config)
    for path,dirs,files in os.walk(config['base_path']):
        if len(dirs) < 1:
            continue
        final_table = pd.DataFrame()
    #         print("number of subflows", subflows_num)
        suffix = "_" + str(subflows_num)
        metrics = {
            'flow_1_SR': [],
            'flow_1_ABU': [],
            'flow_1_LR': [],
            'flow_1_LRN': [],            
            'flow_1_QMD': [],
            'flow_1_GP': [],
            
            'flow_2_SR': [],
            'flow_2_ABU': [],
            'flow_2_LR': [],
            'flow_2_LRN': [],            
            'flow_2_QMD': [],
            'flow_2_GP': [],
            
            'single_SR': [],
            'single_ABU': [],
            'single_LR': [],
            'single_LRN': [],            
            'single_QMD': [],
            'single_GP': [],
        }

        subjects = []
        for directory in dirs:
            subflows_num = 2
            if not directory.endswith(suffix):
                continue
            try:
                subflows = collect_subflows(config['mp_channel'], config['base_path'] + '/' + directory, subflows_num)
            except:
                print("There is an exception occured at:", directory)

            single_flow = SingleFlow(config['single_channel'], config['base_path'] + '/' + directory, 1)

            abs_ABU = sum(subflow.get_summary()['ABU'] for subflow in subflows)
            abs_LR = sum(subflow.get_summary()['LR'] for subflow in subflows)
            abs_GP = sum(subflow.get_summary()['GP'] for subflow in subflows)
            abs_QMD = sum(subflow.get_summary()['QMD'] for subflow in subflows)
            abs_LRN = sum(subflow.get_summary()['LRN'] for subflow in subflows)
            subjects.append((directory, abs_ABU, abs_LR, abs_GP, abs_QMD, abs_LRN))

            metrics['flow_1_SR'].append(subflows[0].get_summary()['SR'])
            metrics['flow_1_ABU'].append(subflows[0].get_summary()['ABU'])            
            metrics['flow_1_LR'].append(subflows[0].get_summary()['LR'])
            metrics['flow_1_LRN'].append(subflows[0].get_summary()['LRN'])            
            metrics['flow_1_QMD'].append(subflows[0].get_summary()['QMD'])
            metrics['flow_1_GP'].append(subflows[0].get_summary()['GP'])
            
            metrics['flow_2_SR'].append(subflows[1].get_summary()['SR'])
            metrics['flow_2_ABU'].append(subflows[1].get_summary()['ABU'])            
            metrics['flow_2_LR'].append(subflows[1].get_summary()['LR'])
            metrics['flow_2_LRN'].append(subflows[1].get_summary()['LRN'])            
            metrics['flow_2_QMD'].append(subflows[1].get_summary()['QMD'])
            metrics['flow_2_GP'].append(subflows[1].get_summary()['GP'])
            
            metrics['single_SR'].append(single_flow.get_summary()['SR'])
            metrics['single_ABU'].append(single_flow.get_summary()['ABU'])            
            metrics['single_LR'].append(single_flow.get_summary()['LR'])
            metrics['single_LRN'].append(single_flow.get_summary()['LRN'])            
            metrics['single_QMD'].append(single_flow.get_summary()['QMD'])
            metrics['single_GP'].append(single_flow.get_summary()['GP'])

        # Display parameters
        if config['debug']:
            index = 1
            if (len(subjects) < MIN_MEASUREMENT_NUMBER):
                print("for measuring ', subflows_num, 'subflows, \
                      it does not have the necessary minimum number of measurements")
#             sorted_list = sorted(subjects, key=lambda x: x[1], reverse=True)
            sorted_list = sorted(subjects, key=lambda x: x[4], reverse=False)
            for item in sorted_list:
                print("Rank:", index)
                print("Directory, ABU, LR, GP, QMD, LRN: ", item)
                pdf = PDF(config['base_path'] + '/' + item[0] + '/' + config['base_path'] + '_aggr_FRACTaL_50ms_0ms.pdf')
                display(pdf)
                index += 1
            print('-' * 20)

        final_table["RTP flow"] = list(range(7))
        final_table["MPRTP subflow 1"] = list(range(7))
        final_table["MPRTP subflow 2"] = list(range(7))        
        flow_1_SRs = np.asarray(metrics['flow_1_SR'])
        flow_1_ABUs = np.asarray(metrics['flow_1_ABU'])
        flow_1_LRs = np.asarray(metrics['flow_1_LR'])
        flow_1_LRNs = np.asarray(metrics['flow_1_LRN'])
        flow_1_QMDs = np.asarray(metrics['flow_1_QMD'])
        flow_1_GPs = np.asarray(metrics['flow_1_GP'])
        
        flow_2_SRs = np.asarray(metrics['flow_2_SR'])
        flow_2_ABUs = np.asarray(metrics['flow_2_ABU'])
        flow_2_LRs = np.asarray(metrics['flow_2_LR'])
        flow_2_LRNs = np.asarray(metrics['flow_2_LRN'])
        flow_2_QMDs = np.asarray(metrics['flow_2_QMD'])
        flow_2_GPs = np.asarray(metrics['flow_2_GP'])
        
        single_SRs = np.asarray(metrics['single_SR'])
        single_ABUs = np.asarray(metrics['single_ABU'])
        single_LRs = np.asarray(metrics['single_LR'])
        single_LRNs = np.asarray(metrics['single_LRN'])
        single_QMDs = np.asarray(metrics['single_QMD'])
        single_GPs = np.asarray(metrics['single_GP'])

        try:
            col = "RTP flow"
            final_table[col][0] = str(int(single_GPs.mean())) + ' ± ' + str(int(single_GPs.std()))
            final_table[col][1] = str(int(single_ABUs.mean() * 100)) + '% ± ' + str(int(single_ABUs.std() * 100) ) + '%'
            final_table[col][2] = str(int(single_LRs.mean() * 100) ) + '% ± ' + str(int(single_LRs.std() * 100)) + '%'
            final_table[col][3] = str(int(single_LRNs.mean())) + ' ± ' + str(int(single_LRNs.std()))        
            final_table[col][4] = str(round(single_QMDs.mean(), 2)) + ' ± ' + str(round(single_QMDs.std(), 2))
            final_table[col][5] = str(int(single_SRs.mean())) + ' ± ' + str(int(single_SRs.std()) )
            final_table[col][6] = str(len(metrics['single_SR']))
            col = "MPRTP subflow 1"
            final_table[col][0] = str(int(flow_1_GPs.mean())) + ' ± ' + str(int(flow_1_GPs.std()))
            final_table[col][1] = str(int(flow_1_ABUs.mean() * 100)) + '% ± ' + str(int(flow_1_ABUs.std() * 100) ) + '%'
            final_table[col][2] = str(int(flow_1_LRs.mean() * 100) ) + '% ± ' + str(int(flow_1_LRs.std() * 100)) + '%'
            final_table[col][3] = str(int(flow_1_LRNs.mean())) + ' ± ' + str(int(flow_1_LRNs.std()))        
            final_table[col][4] = str(round(flow_1_QMDs.mean(), 2)) + ' ± ' + str(round(flow_1_QMDs.std(), 2))
            final_table[col][5] = str(int(flow_1_SRs.mean())) + ' ± ' + str(int(flow_1_SRs.std()) )
            final_table[col][6] = str(len(metrics['flow_1_SR']))
            col = "MPRTP subflow 2"
            final_table[col][0] = str(int(flow_2_GPs.mean())) + ' ± ' + str(int(flow_2_GPs.std()))
            final_table[col][1] = str(int(flow_2_ABUs.mean() * 100)) + '% ± ' + str(int(flow_2_ABUs.std() * 100) ) + '%'
            final_table[col][2] = str(int(flow_2_LRs.mean() * 100) ) + '% ± ' + str(int(flow_2_LRs.std() * 100)) + '%'
            final_table[col][3] = str(int(flow_2_LRNs.mean())) + ' ± ' + str(int(flow_2_LRNs.std()))        
            final_table[col][4] = str(round(flow_2_QMDs.mean(), 2)) + ' ± ' + str(round(flow_2_QMDs.std(), 2))
            final_table[col][5] = str(int(flow_2_SRs.mean())) + ' ± ' + str(int(flow_2_SRs.std()) )
            final_table[col][6] = str(len(metrics['flow_2_SR']))
        except:
            print("Problem in calculations", directory)

        final_table = final_table.rename({
            0: 'GP [kbps]',
            1: 'ABU [%]', 
            2: 'LR [%]', 
            3: 'LRN',
            4: 'QMD [ms]',
            5: 'SR [kbps]',
            6: 'Samples'}, axis='index')
    #     display(final_table)
        display(final_table.loc[['GP [kbps]', 'ABU [%]', 'LR [%]', 'QMD [ms]'], :])

    
# We need the number of lost packets instead!!!!!

{'debug': False, 'mp_channel': '2', 'single_channel': '1', 'base_path': 'mprtp5'}


/usr/local/lib/python2.7/site-packages/ipykernel_launcher.py:257: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
/usr/local/lib/python2.7/site-packages/ipykernel_launcher.py:258: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
/usr/local/lib/python2.7/site-packages/ipykernel_launcher.py:259: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
/usr/local/lib/python2.7/site-packages/ipykernel_launcher.py:260: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

Se

,RTP flow,MPRTP subflow 1,MPRTP subflow 2
GP [kbps],1014 ± 79,427 ± 94,620 ± 126
ABU [%],39% ± 2%,18% ± 3%,25% ± 4%
LR [%],0% ± 0%,0% ± 0%,0% ± 0%
QMD [ms],114.31 ± 2.72,114.13 ± 2.69,114.25 ± 2.78


In [19]:
#######################################################
#      shared bottleneck, fairness test - mprtp7
#######################################################
# TODO!
import pandas as pd
import numpy as np

class PDF(object):
    def __init__(self, pdf, size=(400,200)):
        self.pdf = pdf
        self.size = size

    def _repr_html_(self):
        return '<iframe src={0} width={1[0]} height={1[1]}></iframe>'.format(self.pdf, self.size)

    def _repr_latex_(self):
        return r'\includegraphics[width=1.0\textwidth]{{{0}}}'.format(self.pdf)

packets_colnames = ["Tracked NTP",
                    "Sequence Number",
                    "Timestamp",
                    "SSRC",
                    "Payload Type",
                    "Payload Size",
                    "Subflow ID",
                    "Subflow Sequence Number",
                    "Header Size",
                    "Protect Begin",
                    "Protect End",
                    "Marker",
                   ]

class Subflow:
    def __init__(self, channel, path, subflow_id):
        self._path = path
        self._subflow_id = subflow_id
        self._df = pd.DataFrame()
        sr = os.path.join(path, 'rtpsubflow_'+ str(subflow_id) +'_sr.csv')
        pathbw = os.path.join(path, 'subflow_'+ str(subflow_id) +'_pathbw.csv')
        qmd = os.path.join(path, 'rtpsubflow_'+ str(subflow_id) +'_qmd.csv')
        self._df = pd.concat([
            pd.read_csv(sr, index_col=None, header=None, names=['Sending Rate', 'FEC Rate']),
            pd.read_csv(pathbw, index_col=None, header=None, names=['Capacity']),
            pd.read_csv(qmd, index_col=None, header=None, names=['Queue Delay']),
        ], axis=1)

        snd_csv = os.path.join(path, 'snd_packets_' + channel + '_s'+ str(subflow_id) +'.csv')
        snd_packets = pd.read_csv(snd_csv, index_col=None, header=None, names=packets_colnames)
        snd_packets = snd_packets[snd_packets["Payload Type"] == 96]
        
        ply_csv = os.path.join(path, 'rcv_packets_' + channel + '_s'+ str(subflow_id) +'.csv')
        ply_packets = pd.read_csv(ply_csv, index_col=None, header=None, names=packets_colnames)
        ply_packets = ply_packets[ply_packets["Payload Type"] == 96]
        
        self._df['SR'] = self._df.apply (lambda row: (row['Sending Rate'] + row['FEC Rate']) // 125, axis=1)
        self._df['ABU'] = self._df.apply (lambda row: ((row['Sending Rate'] + row['FEC Rate']) // 125) / row['Capacity'], axis=1)        
        self._df['QMD'] = self._df.apply (lambda row: row['Queue Delay'] // 1000, axis=1)
        self._lr = 1.0 - float(len(ply_packets)) / float(len(snd_packets))
        self._lrn = len(snd_packets) - len(ply_packets)
        gp = os.path.join(path, 'rtpsubflow_'+ str(subflow_id) +'_gp_avg.csv')
        with open(gp, 'r') as content_file:
            self._gp = float(content_file.read()) / 125
        
    @property
    def sr(self):
        return self._sr

    def get_summary(self):
        return {
            'SR': self._df['SR'].mean(),
            'ABU': self._df["ABU"].mean(),
            'QMD': self._df['QMD'].mean(),
            'LR': self._lr,
            'LRN': self._lrn,            
            'GP': self._gp
        }

class SingleFlow:
    def __init__(self, channel, path, flow_id):
        self._path = path
        self._flow_id = flow_id
        self._df = pd.DataFrame()
        sr = os.path.join(path, 'flow_'+ str(flow_id) +'_sr.csv')
        pathbw = os.path.join(path, 'flows_'+ str(flow_id) +'_pathbw.csv')
        qmd = os.path.join(path, 'flow_'+ str(flow_id) +'_qmd.csv')
        self._df = pd.concat([
            pd.read_csv(sr, index_col=None, header=None, names=['Sending Rate', 'FEC Rate']),
            pd.read_csv(pathbw, index_col=None, header=None, names=['Capacity']),
            pd.read_csv(qmd, index_col=None, header=None, names=['Queue Delay']),
        ], axis=1)

        snd_csv = os.path.join(path, 'snd_packets_' + channel + '.csv')
        snd_packets = pd.read_csv(snd_csv, index_col=None, header=None, names=packets_colnames)
        snd_packets = snd_packets[snd_packets["Payload Type"] == 96]
        
        ply_csv = os.path.join(path, 'rcv_packets_' + channel +'.csv')
        ply_packets = pd.read_csv(ply_csv, index_col=None, header=None, names=packets_colnames)
        ply_packets = ply_packets[ply_packets["Payload Type"] == 96]
        
        self._df['SR'] = self._df.apply (lambda row: (row['Sending Rate'] + row['FEC Rate']) // 125, axis=1)
        self._df['ABU'] = self._df.apply (lambda row: ((row['Sending Rate'] + row['FEC Rate']) // 125) / row['Capacity'], axis=1)        
        self._df['QMD'] = self._df.apply (lambda row: row['Queue Delay'] // 1000, axis=1)
        self._lr = 1.0 - float(len(ply_packets)) / float(len(snd_packets))
        self._lrn = len(snd_packets) - len(ply_packets)
        gp = os.path.join(path, 'flow_'+ str(flow_id) +'_gp_avg.csv')
        with open(gp, 'r') as content_file:
            self._gp = float(content_file.read()) / 125
        
    @property
    def sr(self):
        return self._sr

    def get_summary(self):
        return {
            'SR': self._df['SR'].mean(),
            'ABU': self._df["ABU"].mean(),
            'QMD': self._df['QMD'].mean(),
            'LR': self._lr,
            'LRN': self._lrn,            
            'GP': self._gp
        }
        
def collect_subflows(channel, path, subflows_num):
    subflows = [Subflow(channel, path, subflow_id) for subflow_id in range(1, subflows_num + 1)]
    return subflows

    
import os
from os import listdir
from os.path import isfile, join

configs = [
{
    'base_path': "mprtp7",
    'debug': False,
#     'debug': True,
    'mp_channel': "2",
    'single_channel_1': "1",
    'single_channel_2': "3",
}, 
]

for config in configs:
    print(config)
    for path,dirs,files in os.walk(config['base_path']):
        if len(dirs) < 1:
            continue
        final_table = pd.DataFrame()
    #         print("number of subflows", subflows_num)
        suffix = "_" + str(subflows_num)
        metrics = {
            'SR': [],
            'ABU': [],
            'LR': [],
            'LRN': [],            
            'QMD': [],
            'GP': [],
            
            'single_1_SR': [],
            'single_1_ABU': [],
            'single_1_LR': [],
            'single_1_LRN': [],            
            'single_1_QMD': [],
            'single_1_GP': [],
            
            'single_2_SR': [],
            'single_2_ABU': [],
            'single_2_LR': [],
            'single_2_LRN': [],            
            'single_2_QMD': [],
            'single_2_GP': [],
        }

        subjects = []
        for directory in dirs:
            subflows_num = 2
            if not directory.endswith(suffix):
                continue
            try:
                subflows = collect_subflows(config['mp_channel'], config['base_path'] + '/' + directory, subflows_num)
            except:
                print("There is an exception occured at:", directory)

            single_flow_1 = SingleFlow(config['single_channel_1'], config['base_path'] + '/' + directory, 1)
            single_flow_2 = SingleFlow(config['single_channel_2'], config['base_path'] + '/' + directory, 2)

            abs_ABU = sum(subflow.get_summary()['ABU'] for subflow in subflows)
            abs_LR = sum(subflow.get_summary()['LR'] for subflow in subflows)
            abs_GP = sum(subflow.get_summary()['GP'] for subflow in subflows)
            abs_QMD = sum(subflow.get_summary()['QMD'] for subflow in subflows)
            abs_LRN = sum(subflow.get_summary()['LRN'] for subflow in subflows)
            subjects.append((directory, abs_ABU, abs_LR, abs_GP, abs_QMD, abs_LRN))

            SR = sum(subflow.get_summary()['SR'] for subflow in subflows) 
            ABU = sum(subflow.get_summary()['ABU'] for subflow in subflows) 
            LR = sum(subflow.get_summary()['LR'] for subflow in subflows) / float(subflows_num)
            LRN = sum(subflow.get_summary()['LRN'] for subflow in subflows)           
            QMD = sum(subflow.get_summary()['QMD'] for subflow in subflows) / float(subflows_num)
            GP = sum(subflow.get_summary()['GP'] for subflow in subflows)

            metrics['SR'].append(SR)
            metrics['ABU'].append(ABU)            
            metrics['LR'].append(LR)
            metrics['LRN'].append(LRN)            
            metrics['QMD'].append(QMD)
            metrics['GP'].append(GP)
            
            metrics['single_1_SR'].append(single_flow_1.get_summary()['SR'])
            metrics['single_1_ABU'].append(single_flow_1.get_summary()['ABU'])            
            metrics['single_1_LR'].append(single_flow_1.get_summary()['LR'])
            metrics['single_1_LRN'].append(single_flow_1.get_summary()['LRN'])            
            metrics['single_1_QMD'].append(single_flow_1.get_summary()['QMD'])
            metrics['single_1_GP'].append(single_flow_1.get_summary()['GP'])
            
            metrics['single_2_SR'].append(single_flow_2.get_summary()['SR'])
            metrics['single_2_ABU'].append(single_flow_2.get_summary()['ABU'])            
            metrics['single_2_LR'].append(single_flow_2.get_summary()['LR'])
            metrics['single_2_LRN'].append(single_flow_2.get_summary()['LRN'])            
            metrics['single_2_QMD'].append(single_flow_2.get_summary()['QMD'])
            metrics['single_2_GP'].append(single_flow_2.get_summary()['GP'])

        # Display parameters
        if config['debug']:
            index = 1
            if (len(subjects) < MIN_MEASUREMENT_NUMBER):
                print("for measuring ', subflows_num, 'subflows, \
                      it does not have the necessary minimum number of measurements")
#             sorted_list = sorted(subjects, key=lambda x: x[1], reverse=True)
            sorted_list = sorted(subjects, key=lambda x: x[4], reverse=False)
            for item in sorted_list:
                print("Rank:", index)
                print("Directory, ABU, LR, GP, QMD, LRN: ", item)
                pdf = PDF(config['base_path'] + '/' + item[0] + '/' + config['base_path'] + '_aggr_FRACTaL_50ms_0ms.pdf')
                display(pdf)
                index += 1
            print('-' * 20)

        final_table["RTP flow 1"] = list(range(7))
        final_table["RTP flow 2"] = list(range(7))        
        final_table["MPRTP flow"] = list(range(7))
        SRs = np.asarray(metrics['SR'])
        ABUs = np.asarray(metrics['ABU'])
        LRs = np.asarray(metrics['LR'])
        LRNs = np.asarray(metrics['LRN'])
        QMDs = np.asarray(metrics['QMD'])
        GPs = np.asarray(metrics['GP'])
        
        single_1_SRs = np.asarray(metrics['single_1_SR'])
        single_1_ABUs = np.asarray(metrics['single_1_ABU'])
        single_1_LRs = np.asarray(metrics['single_1_LR'])
        single_1_LRNs = np.asarray(metrics['single_1_LRN'])
        single_1_QMDs = np.asarray(metrics['single_1_QMD'])
        single_1_GPs = np.asarray(metrics['single_1_GP'])
        
        single_2_SRs = np.asarray(metrics['single_2_SR'])
        single_2_ABUs = np.asarray(metrics['single_2_ABU'])
        single_2_LRs = np.asarray(metrics['single_2_LR'])
        single_2_LRNs = np.asarray(metrics['single_2_LRN'])
        single_2_QMDs = np.asarray(metrics['single_2_QMD'])
        single_2_GPs = np.asarray(metrics['single_2_GP'])

        try:
            col = "RTP flow 1"
            final_table[col][0] = str(int(single_1_GPs.mean())) + ' ± ' + str(int(single_1_GPs.std()))
            final_table[col][1] = str(int(single_1_ABUs.mean() * 100)) + '% ± ' + str(int(single_1_ABUs.std() * 100) ) + '%'
            final_table[col][2] = str(int(single_1_LRs.mean() * 100) ) + '% ± ' + str(int(single_1_LRs.std() * 100)) + '%'
            final_table[col][3] = str(int(single_1_LRNs.mean())) + ' ± ' + str(int(single_1_LRNs.std()))        
            final_table[col][4] = str(round(single_1_QMDs.mean(), 2)) + ' ± ' + str(round(single_1_QMDs.std(), 2))
            final_table[col][5] = str(int(single_1_SRs.mean())) + ' ± ' + str(int(single_1_SRs.std()) )
            final_table[col][6] = str(len(metrics['single_1_SR']))
            col = "RTP flow 2"
            final_table[col][0] = str(int(single_2_GPs.mean())) + ' ± ' + str(int(single_2_GPs.std()))
            final_table[col][1] = str(int(single_2_ABUs.mean() * 100)) + '% ± ' + str(int(single_2_ABUs.std() * 100) ) + '%'
            final_table[col][2] = str(int(single_2_LRs.mean() * 100) ) + '% ± ' + str(int(single_2_LRs.std() * 100)) + '%'
            final_table[col][3] = str(int(single_2_LRNs.mean())) + ' ± ' + str(int(single_2_LRNs.std()))        
            final_table[col][4] = str(round(single_2_QMDs.mean(), 2)) + ' ± ' + str(round(single_2_QMDs.std(), 2))
            final_table[col][5] = str(int(single_2_SRs.mean())) + ' ± ' + str(int(single_2_SRs.std()) )
            final_table[col][6] = str(len(metrics['single_2_SR']))
            col = "MPRTP flow"
            final_table[col][0] = str(int(GPs.mean())) + ' ± ' + str(int(GPs.std()))
            final_table[col][1] = str(int(ABUs.mean() * 100)) + '% ± ' + str(int(ABUs.std() * 100) ) + '%'
            final_table[col][2] = str(int(LRs.mean() * 100) ) + '% ± ' + str(int(LRs.std() * 100)) + '%'
            final_table[col][3] = str(int(LRNs.mean())) + ' ± ' + str(int(LRNs.std()))        
            final_table[col][4] = str(round(QMDs.mean(), 2)) + ' ± ' + str(round(QMDs.std(), 2))
            final_table[col][5] = str(int(SRs.mean())) + ' ± ' + str(int(SRs.std()) )
            final_table[col][6] = str(len(metrics['SR']))
        except:
            print("Problem in calculations", directory)

        final_table = final_table.rename({
            0: 'GP [kbps]',
            1: 'ABU [%]', 
            2: 'LR [%]', 
            3: 'LRN',
            4: 'QMD [ms]',
            5: 'SR [kbps]',
            6: 'Samples'}, axis='index')
    #     display(final_table)
        display(final_table.loc[['GP [kbps]', 'LR [%]', 'QMD [ms]'], :])

    
# We need the number of lost packets instead!!!!!

{'debug': False, 'mp_channel': '2', 'single_channel_1': '1', 'base_path': 'mprtp7', 'single_channel_2': '3'}


/usr/local/lib/python2.7/site-packages/ipykernel_launcher.py:264: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
/usr/local/lib/python2.7/site-packages/ipykernel_launcher.py:265: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
/usr/local/lib/python2.7/site-packages/ipykernel_launcher.py:266: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
/usr/local/lib/python2.7/site-packages/ipykernel_launcher.py:267: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

Se

,RTP flow 1,RTP flow 2,MPRTP flow
GP [kbps],940 ± 38,965 ± 44,828 ± 39
LR [%],0% ± 0%,0% ± 0%,0% ± 0%
QMD [ms],108.41 ± 0.5,109.52 ± 2.2,107.75 ± 1.05


In [ ]:
# mprtp 10 - 3 paths, 2 mprtp 2 subflows
# mprtp 11 - 2 path, 2 mprtp 2 subflows

In [14]:
#######################################################
#      shared bottleneck, fairness test - mprtp11
#######################################################
# TODO!
import pandas as pd
import numpy as np

class PDF(object):
    def __init__(self, pdf, size=(400,200)):
        self.pdf = pdf
        self.size = size

    def _repr_html_(self):
        return '<iframe src={0} width={1[0]} height={1[1]}></iframe>'.format(self.pdf, self.size)

    def _repr_latex_(self):
        return r'\includegraphics[width=1.0\textwidth]{{{0}}}'.format(self.pdf)

packets_colnames = ["Tracked NTP",
                    "Sequence Number",
                    "Timestamp",
                    "SSRC",
                    "Payload Type",
                    "Payload Size",
                    "Subflow ID",
                    "Subflow Sequence Number",
                    "Header Size",
                    "Protect Begin",
                    "Protect End",
                    "Marker",
                   ]

class Subflow:
    def __init__(self, channel, path, subflow_id, flow_id, path_id):
        self._path = path
        self._subflow_id = subflow_id
        self._df = pd.DataFrame()
        # mprtpflow_1_flow_1_path_1_sr
        sr = os.path.join(path, 'mprtpflow_' + str(channel) +
                          '_flow_' + str(flow_id) + '_path_' + str(path_id) + '_sr.csv')
        pathbw = os.path.join(path, 'path_' + str(path_id) + '_pathbw' + '.csv')
        qmd = os.path.join(path, 'mprtpflow_' + str(channel) +
                          '_flow_' + str(flow_id) + '_path_' + str(path_id) + '_qmd.csv')
        self._df = pd.concat([
            pd.read_csv(sr, index_col=None, header=None, names=['Sending Rate', 'FEC Rate']),
            pd.read_csv(pathbw, index_col=None, header=None, names=['Capacity']),
            pd.read_csv(qmd, index_col=None, header=None, names=['Queue Delay']),
        ], axis=1)

        snd_csv = os.path.join(path, 'snd_packets_' + channel + '_s'+ str(subflow_id) +'.csv')
        snd_packets = pd.read_csv(snd_csv, index_col=None, header=None, names=packets_colnames)
        snd_packets = snd_packets[snd_packets["Payload Type"] == 96]
        
        ply_csv = os.path.join(path, 'rcv_packets_' + channel + '_s'+ str(subflow_id) +'.csv')
        ply_packets = pd.read_csv(ply_csv, index_col=None, header=None, names=packets_colnames)
        ply_packets = ply_packets[ply_packets["Payload Type"] == 96]
        
        self._df['SR'] = self._df.apply (lambda row: (row['Sending Rate'] + row['FEC Rate']) // 125, axis=1)
        self._df['ABU'] = self._df.apply (lambda row: ((row['Sending Rate'] + row['FEC Rate']) // 125) / row['Capacity'], axis=1)        
        self._df['QMD'] = self._df.apply (lambda row: row['Queue Delay'] // 1000, axis=1)
        self._lr = 1.0 - float(len(ply_packets)) / float(len(snd_packets))
        self._lrn = len(snd_packets) - len(ply_packets)
        gp = os.path.join(path, 'mprtpflow_' + str(channel) +
                          '_flow_' + str(flow_id) + '_path_' + str(path_id) + '_gp_avg.csv')
        with open(gp, 'r') as content_file:
            self._gp = float(content_file.read()) / 125
        
    @property
    def sr(self):
        return self._sr

    def get_summary(self):
        return {
            'SR': self._df['SR'].mean(),
            'ABU': self._df["ABU"].mean(),
            'QMD': self._df['QMD'].mean(),
            'LR': self._lr,
            'LRN': self._lrn,            
            'GP': self._gp
        }

def collect_subflows(channel, path, subflows_num, flow_id, path_id):
    subflows = [Subflow(channel, path, subflow_id, flow_id, path_id) for subflow_id in range(1, subflows_num + 1)]
    return subflows

    
import os
from os import listdir
from os.path import isfile, join

configs = [
{
    'base_path': "mprtp11",
    'debug': False,
#     'debug': True,
    'mp_channel': "1",
    'mp_channel_2': "2",
}, 
]

for config in configs:
    print(config)
    for path,dirs,files in os.walk(config['base_path']):
        if len(dirs) < 1:
            continue
        final_table = pd.DataFrame()
    #         print("number of subflows", subflows_num)
        suffix = "_" + str(subflows_num)
        metrics = {
            'SR': [],
            'ABU': [],
            'LR': [],
            'LRN': [],            
            'QMD': [],
            'GP': [],

            'SR_2': [],
            'ABU_2': [],
            'LR_2': [],
            'LRN_2': [],            
            'QMD_2': [],
            'GP_2': [],
        }

        subjects = []
        for directory in dirs:
            subflows_num = 2
            if not directory.endswith(suffix):
                continue
            try:
                subflows = collect_subflows(config['mp_channel'], config['base_path'] + '/' + directory, subflows_num, 
                                            flow_id=1, path_id=1)
                subflows_2 = collect_subflows(config['mp_channel_2'], config['base_path'] + '/' + directory, subflows_num, 
                                            flow_id=2, path_id=2)

                SR = sum(subflow.get_summary()['SR'] for subflow in subflows) 
                ABU = sum(subflow.get_summary()['ABU'] for subflow in subflows) 
                LR = sum(subflow.get_summary()['LR'] for subflow in subflows) / float(subflows_num)
                LRN = sum(subflow.get_summary()['LRN'] for subflow in subflows)           
                QMD = sum(subflow.get_summary()['QMD'] for subflow in subflows) / float(subflows_num)
                GP = sum(subflow.get_summary()['GP'] for subflow in subflows)

                SR_2 = sum(subflow.get_summary()['SR'] for subflow in subflows_2) 
                ABU_2 = sum(subflow.get_summary()['ABU'] for subflow in subflows_2) 
                LR_2 = sum(subflow.get_summary()['LR'] for subflow in subflows_2) / float(subflows_num)
                LRN_2 = sum(subflow.get_summary()['LRN'] for subflow in subflows_2)           
                QMD_2 = sum(subflow.get_summary()['QMD'] for subflow in subflows_2) / float(subflows_num)
                GP_2 = sum(subflow.get_summary()['GP'] for subflow in subflows_2)
                
                metrics['SR'].append(SR)
                metrics['ABU'].append(ABU)            
                metrics['LR'].append(LR)
                metrics['LRN'].append(LRN)            
                metrics['QMD'].append(QMD)
                metrics['GP'].append(GP)

                metrics['SR_2'].append(SR_2)
                metrics['ABU_2'].append(ABU_2)            
                metrics['LR_2'].append(LR_2)
                metrics['LRN_2'].append(LRN_2)            
                metrics['QMD_2'].append(QMD_2)
                metrics['GP_2'].append(GP_2)
            except Exception, e:
                print("There is an exception occured at:", directory, "Error message:", e)
                

        # Display parameters
        if config['debug']:
            index = 1
            if (len(subjects) < MIN_MEASUREMENT_NUMBER):
                print("for measuring ', subflows_num, 'subflows, \
                      it does not have the necessary minimum number of measurements")
#             sorted_list = sorted(subjects, key=lambda x: x[1], reverse=True)
            sorted_list = sorted(subjects, key=lambda x: x[4], reverse=False)
            for item in sorted_list:
                print("Rank:", index)
                print("Directory, ABU, LR, GP, QMD, LRN: ", item)
                pdf = PDF(config['base_path'] + '/' + item[0] + '/' + config['base_path'] + '_aggr_FRACTaL_50ms_0ms.pdf')
                display(pdf)
                index += 1
            print('-' * 20)

        final_table["MPRTP Flows 1"] = list(range(7))
        final_table["MPRTP Flows 2"] = list(range(7))        
        SRs = np.asarray(metrics['SR'])
        ABUs = np.asarray(metrics['ABU'])
        LRs = np.asarray(metrics['LR'])
        LRNs = np.asarray(metrics['LRN'])
        QMDs = np.asarray(metrics['QMD'])
        GPs = np.asarray(metrics['GP'])
        
        SRs_2 = np.asarray(metrics['SR_2'])
        ABUs_2 = np.asarray(metrics['ABU_2'])
        LRs_2 = np.asarray(metrics['LR_2'])
        LRNs_2 = np.asarray(metrics['LRN_2'])
        QMDs_2 = np.asarray(metrics['QMD_2'])
        GPs_2 = np.asarray(metrics['GP_2'])
        
        try:
            col = "MPRTP Flows 1"
            final_table[col][0] = str(int(GPs.mean())) + ' ± ' + str(int(GPs.std()))
            final_table[col][1] = str(1000 * subflows_num)
            final_table[col][2] = str(int(ABUs.mean() * 100)) + '% ± ' + str(int(ABUs.std() * 100) ) + '%'
            final_table[col][3] = str(int(LRs.mean() * 100) ) + '% ± ' + str(int(LRs.std() * 100)) + '%'
            final_table[col][4] = str(int(LRNs.mean())) + ' ± ' + str(int(LRNs.std()))        
            final_table[col][5] = str(round(QMDs.mean(), 2)) + ' ± ' + str(round(QMDs.std(), 2))
            final_table[col][6] = str(int(SRs.mean())) + ' ± ' + str(int(SRs.std()) )
            final_table[col][7] = str(len(metrics['SR']))
            
            col = "MPRTP Flows 2"
            final_table[col][0] = str(int(GPs_2.mean())) + ' ± ' + str(int(GPs_2.std()))
            final_table[col][1] = str(1000 * subflows_num)
            final_table[col][2] = str(int(ABUs_2.mean() * 100)) + '% ± ' + str(int(ABUs_2.std() * 100) ) + '%'
            final_table[col][3] = str(int(LRs_2.mean() * 100) ) + '% ± ' + str(int(LRs_2.std() * 100)) + '%'
            final_table[col][4] = str(int(LRNs_2.mean())) + ' ± ' + str(int(LRNs_2.std()))        
            final_table[col][5] = str(round(QMDs_2.mean(), 2)) + ' ± ' + str(round(QMDs_2.std(), 2))
            final_table[col][6] = str(int(SRs_2.mean())) + ' ± ' + str(int(SRs_2.std()))
            final_table[col][7] = str(len(metrics['SR_2']))
        except:
            print("Problem in calculations", directory)

        final_table = final_table.rename({
            0: 'GP [kbps]',
            1: 'Σ [kbps]', 
            2: 'ABU [%]',
            3: 'LR [%]', 
            4: 'LRN',
            5: 'QMD [ms]',
            6: 'SR [kbps]',
            7: 'Samples'}, axis='index')
    #     display(final_table)
        display(final_table.loc[['Σ [kbps]', 'GP [kbps]', 'LR [%]', 'QMD [ms]'], :])

    
# We need the number of lost packets instead!!!!!

{'debug': False, 'mp_channel': '1', 'mp_channel_2': '2', 'base_path': 'mprtp11'}


/usr/local/lib/python2.7/site-packages/ipykernel_launcher.py:201: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
/usr/local/lib/python2.7/site-packages/ipykernel_launcher.py:202: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
/usr/local/lib/python2.7/site-packages/ipykernel_launcher.py:203: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
/usr/local/lib/python2.7/site-packages/ipykernel_launcher.py:204: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

Se

,MPRTP Flows 1,MPRTP Flows 2
Σ [kbps],2000,2000
GP [kbps],559 ± 17,628 ± 30
LR [%],0% ± 0%,0% ± 0%
QMD [ms],106.08 ± 0.65,108.23 ± 1.09


In [21]:
#######################################################
#      shared bottleneck, fairness test - mprtp10
#######################################################
# TODO!
import pandas as pd
import numpy as np

class PDF(object):
    def __init__(self, pdf, size=(400,200)):
        self.pdf = pdf
        self.size = size

    def _repr_html_(self):
        return '<iframe src={0} width={1[0]} height={1[1]}></iframe>'.format(self.pdf, self.size)

    def _repr_latex_(self):
        return r'\includegraphics[width=1.0\textwidth]{{{0}}}'.format(self.pdf)

packets_colnames = ["Tracked NTP",
                    "Sequence Number",
                    "Timestamp",
                    "SSRC",
                    "Payload Type",
                    "Payload Size",
                    "Subflow ID",
                    "Subflow Sequence Number",
                    "Header Size",
                    "Protect Begin",
                    "Protect End",
                    "Marker",
                   ]

class Subflow:
    def __init__(self, channel, path, subflow_id, flow_id, path_id):
        self._path = path
        self._subflow_id = subflow_id
        self._df = pd.DataFrame()
        # mprtpflow_1_flow_1_path_1_sr
        sr = os.path.join(path, 'mprtpflow_' + str(channel) +
                          '_flow_' + str(flow_id) + '_path_' + str(path_id) + '_sr.csv')
        pathbw = os.path.join(path, 'path_' + str(path_id) + '_pathbw' + '.csv')
        qmd = os.path.join(path, 'mprtpflow_' + str(channel) +
                          '_flow_' + str(flow_id) + '_path_' + str(path_id) + '_qmd.csv')
        self._df = pd.concat([
            pd.read_csv(sr, index_col=None, header=None, names=['Sending Rate', 'FEC Rate']),
            pd.read_csv(pathbw, index_col=None, header=None, names=['Capacity']),
            pd.read_csv(qmd, index_col=None, header=None, names=['Queue Delay']),
        ], axis=1)

        snd_csv = os.path.join(path, 'snd_packets_' + channel + '_s'+ str(subflow_id) +'.csv')
        snd_packets = pd.read_csv(snd_csv, index_col=None, header=None, names=packets_colnames)
        snd_packets = snd_packets[snd_packets["Payload Type"] == 96]
        
        ply_csv = os.path.join(path, 'rcv_packets_' + channel + '_s'+ str(subflow_id) +'.csv')
        ply_packets = pd.read_csv(ply_csv, index_col=None, header=None, names=packets_colnames)
        ply_packets = ply_packets[ply_packets["Payload Type"] == 96]
        
        self._df['SR'] = self._df.apply (lambda row: (row['Sending Rate'] + row['FEC Rate']) // 125, axis=1)
        self._df['ABU'] = self._df.apply (lambda row: ((row['Sending Rate'] + row['FEC Rate']) // 125) / row['Capacity'], axis=1)        
        self._df['QMD'] = self._df.apply (lambda row: row['Queue Delay'] // 1000, axis=1)
        self._lr = 1.0 - float(len(ply_packets)) / float(len(snd_packets))
        self._lrn = len(snd_packets) - len(ply_packets)
        gp = os.path.join(path, 'mprtpflow_' + str(channel) +
                          '_flow_' + str(flow_id) + '_path_' + str(path_id) + '_gp_avg.csv')
        with open(gp, 'r') as content_file:
            self._gp = float(content_file.read()) / 125
        
    @property
    def sr(self):
        return self._sr

    def get_summary(self):
        return {
            'SR': self._df['SR'].mean(),
            'ABU': self._df["ABU"].mean(),
            'QMD': self._df['QMD'].mean(),
            'LR': self._lr,
            'LRN': self._lrn,            
            'GP': self._gp
        }

def collect_subflows(channel, path, subflows_num, flow_ids, path_ids):
    subflows = []
    subflow_id = 0
    for path_id in path_ids:
        flow_id = flow_ids[subflow_id]
        subflow_id += 1
        subflow = Subflow(channel, path, subflow_id, flow_id, path_id)
        subflows.append(subflow)
    return subflows

    
import os
from os import listdir
from os.path import isfile, join

configs = [
{
    'base_path': "mprtp10",
#     'debug': False,
    'debug': True,
    'mp_channel': "1",
    'mp_channel_2': "2",
}, 
]
subflows_num = 2

for config in configs:
    print(config)
    for path,dirs,files in os.walk(config['base_path']):
        if len(dirs) < 1:
            continue
        final_table = pd.DataFrame()
    #         print("number of subflows", subflows_num)
        suffix = "_" + str(subflows_num)
        metrics = {
            'SR': [],
            'ABU': [],
            'LR': [],
            'LRN': [],            
            'QMD': [],
            'GP': [],

            'SR_2': [],
            'ABU_2': [],
            'LR_2': [],
            'LRN_2': [],            
            'QMD_2': [],
            'GP_2': [],
        }

        subjects = []
        for directory in dirs:
            subflows_num = 2
            if not directory.endswith(suffix):
                continue
            try:
                subflows = collect_subflows(config['mp_channel'], config['base_path'] + '/' + directory, subflows_num, 
                                            flow_ids=[1,2], path_ids=[1,2])
                subflows_2 = collect_subflows(config['mp_channel_2'], config['base_path'] + '/' + directory, subflows_num, 
                                            flow_ids=[1,2], path_ids=[2,3])

                SR = sum(subflow.get_summary()['SR'] for subflow in subflows) 
                ABU = sum(subflow.get_summary()['ABU'] for subflow in subflows) 
                LR = sum(subflow.get_summary()['LR'] for subflow in subflows) / float(subflows_num)
                LRN = sum(subflow.get_summary()['LRN'] for subflow in subflows)           
                QMD = sum(subflow.get_summary()['QMD'] for subflow in subflows) / float(subflows_num)
                GP = sum(subflow.get_summary()['GP'] for subflow in subflows)

                SR_2 = sum(subflow.get_summary()['SR'] for subflow in subflows_2) 
                ABU_2 = sum(subflow.get_summary()['ABU'] for subflow in subflows_2) 
                LR_2 = sum(subflow.get_summary()['LR'] for subflow in subflows_2) / float(subflows_num)
                LRN_2 = sum(subflow.get_summary()['LRN'] for subflow in subflows_2)           
                QMD_2 = sum(subflow.get_summary()['QMD'] for subflow in subflows_2) / float(subflows_num)
                GP_2 = sum(subflow.get_summary()['GP'] for subflow in subflows_2)
                
                metrics['SR'].append(SR)
                metrics['ABU'].append(ABU)            
                metrics['LR'].append(LR)
                metrics['LRN'].append(LRN)            
                metrics['QMD'].append(QMD)
                metrics['GP'].append(GP)

                metrics['SR_2'].append(SR_2)
                metrics['ABU_2'].append(ABU_2)            
                metrics['LR_2'].append(LR_2)
                metrics['LRN_2'].append(LRN_2)            
                metrics['QMD_2'].append(QMD_2)
                metrics['GP_2'].append(GP_2)
            except Exception, e:
                print("There is an exception occured at:", directory, "Error message:", e)
                

        # Display parameters
        if config['debug']:
            index = 1
            if (len(subjects) < MIN_MEASUREMENT_NUMBER):
                print("for measuring ', subflows_num, 'subflows, \
                      it does not have the necessary minimum number of measurements")
#             sorted_list = sorted(subjects, key=lambda x: x[1], reverse=True)
            sorted_list = sorted(subjects, key=lambda x: x[4], reverse=False)
            for item in sorted_list:
                print("Rank:", index)
                print("Directory, ABU, LR, GP, QMD, LRN: ", item)
                pdf = PDF(config['base_path'] + '/' + item[0] + '/' + config['base_path'] + '_aggr_FRACTaL_50ms_0ms.pdf')
                display(pdf)
                index += 1
            print('-' * 20)

        final_table["MPRTP Flows 1"] = list(range(7))
        final_table["MPRTP Flows 2"] = list(range(7))        
        SRs = np.asarray(metrics['SR'])
        ABUs = np.asarray(metrics['ABU'])
        LRs = np.asarray(metrics['LR'])
        LRNs = np.asarray(metrics['LRN'])
        QMDs = np.asarray(metrics['QMD'])
        GPs = np.asarray(metrics['GP'])
        
        SRs_2 = np.asarray(metrics['SR_2'])
        ABUs_2 = np.asarray(metrics['ABU_2'])
        LRs_2 = np.asarray(metrics['LR_2'])
        LRNs_2 = np.asarray(metrics['LRN_2'])
        QMDs_2 = np.asarray(metrics['QMD_2'])
        GPs_2 = np.asarray(metrics['GP_2'])
        
        try:
            col = "MPRTP Flows 1"
            final_table[col][0] = str(int(GPs.mean())) + ' ± ' + str(int(GPs.std()))
            final_table[col][1] = str(1000 * subflows_num)
            final_table[col][2] = str(int(ABUs.mean() * 100)) + '% ± ' + str(int(ABUs.std() * 100) ) + '%'
            final_table[col][3] = str(int(LRs.mean() * 100) ) + '% ± ' + str(int(LRs.std() * 100)) + '%'
            final_table[col][4] = str(int(LRNs.mean())) + ' ± ' + str(int(LRNs.std()))        
            final_table[col][5] = str(round(QMDs.mean(), 2)) + ' ± ' + str(round(QMDs.std(), 2))
            final_table[col][6] = str(int(SRs.mean())) + ' ± ' + str(int(SRs.std()) )
            final_table[col][7] = str(len(metrics['SR']))
            
            col = "MPRTP Flows 2"
            final_table[col][0] = str(int(GPs_2.mean())) + ' ± ' + str(int(GPs_2.std()))
            final_table[col][1] = str(1000 * subflows_num)
            final_table[col][2] = str(int(ABUs_2.mean() * 100)) + '% ± ' + str(int(ABUs_2.std() * 100) ) + '%'
            final_table[col][3] = str(int(LRs_2.mean() * 100) ) + '% ± ' + str(int(LRs_2.std() * 100)) + '%'
            final_table[col][4] = str(int(LRNs_2.mean())) + ' ± ' + str(int(LRNs_2.std()))        
            final_table[col][5] = str(round(QMDs_2.mean(), 2)) + ' ± ' + str(round(QMDs_2.std(), 2))
            final_table[col][6] = str(int(SRs_2.mean())) + ' ± ' + str(int(SRs_2.std()))
            final_table[col][7] = str(len(metrics['SR_2']))
        except:
            print("Problem in calculations", directory)

        final_table = final_table.rename({
            0: 'GP [kbps]',
            1: 'Σ [kbps]',
            2: 'ABU [%]', 
            3: 'LR [%]', 
            4: 'LRN',
            5: 'QMD [ms]',
            6: 'SR [kbps]',
            7: 'Samples'}, axis='index')
    #     display(final_table)
        display(final_table.loc[['Σ [kbps]', 'GP [kbps]', 'LR [%]', 'QMD [ms]'], :])

    
# We need the number of lost packets instead!!!!!

{'debug': True, 'mp_channel': '1', 'mp_channel_2': '2', 'base_path': 'mprtp10'}
for measuring ', subflows_num, 'subflows,                       it does not have the necessary minimum number of measurements
--------------------


/usr/local/lib/python2.7/site-packages/ipykernel_launcher.py:208: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
/usr/local/lib/python2.7/site-packages/ipykernel_launcher.py:209: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
/usr/local/lib/python2.7/site-packages/ipykernel_launcher.py:210: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
/usr/local/lib/python2.7/site-packages/ipykernel_launcher.py:211: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

Se

,MPRTP Flows 1,MPRTP Flows 2
Σ [kbps],2000,2000
GP [kbps],802 ± 82,807 ± 43
LR [%],0% ± 0%,0% ± 0%
QMD [ms],104.68 ± 0.9,104.15 ± 0.52
